<h2 align=center> Classify Radio Signals from Outer Space with Keras</h2>

![](Allen_Telescope.jpg)
[Allen Telescope Array](https://flickr.com/photos/93452909@N00/5656086917) by [brewbooks](https://www.flickr.com/people/93452909@N00) is licensed under [CC BY 2.0](https://creativecommons.org/licenses/by/2.0/)\
[Coursera Link](https://www.coursera.org/projects/classify-radio-signals-space-keras-cnn)\
Create virtual environment\
Unzip dataset.zip\
In terminal, run:\
    - .venv\Scripts\Activate.ps1\
    - pip install --upgrade pip ipykernel scikit-learn tensorflow pandas numpy matplotlib seaborn livelossplot\
    - pip freeze > requirements.txt\

### Training a Convolutional Neural Network (CNN) model to classify space signals into four categories: "squiggle", "narrowband", "noise", and "narrowbanddrd".

## Step 1: Import Libraries

In [ ]:
import warnings;warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
np.random.seed(42)

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

import tensorflow as tf
from livelossplot.tf_keras import PlotLossesCallback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
print('Tensorflow version:', tf.__version__)

%matplotlib inline
print('Tensorflow version:', tf.__version__)

## Step 2: Load and Preprocess SETI Data (signals have been transformed to images)

In [ ]:
train_images = pd.read_csv('dataset/train/images.csv', header=None)
train_labels = pd.read_csv('dataset/train/labels.csv', header=None)

x_train = train_images.values.reshape(3200, 64, 128, 1)
y_train = train_labels.values

In [ ]:
val_images = pd.read_csv('dataset/validation/images.csv', header=None)
val_labels = pd.read_csv('dataset/validation/labels.csv', header=None)

x_val = val_images.values.reshape(800, 64, 128, 1)
y_val = val_labels.values

## Step 3: Plot 2D Spectrograms

In [ ]:
plt.figure(0, figsize=(12,12))
for i in range(1,4):
    plt.subplot(1,3,i)
    img = np.squeeze(x_train[np.random.randint(0, x_train.shape[0])])
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img, cmap="gray")

In [ ]:
plt.imshow(np.squeeze(x_train[3]), cmap="gray")

## Step 4: Create Training and Validation Data Generators

In [ ]:
dataGen_train = ImageDataGenerator(horizontal_flip=True, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2)
dataGen_train.fit(x_train)

dataGen_val = ImageDataGenerator(horizontal_flip=True)
dataGen_val.fit(x_val)

## Step 5: CNN Architecture

In [ ]:
img_shape = (64, 128, 1)
num_classes = 4

model = Sequential([
    ##-> 1st Convolution Layer
    Conv2D(32, (5, 5), padding='same', input_shape=img_shape),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    ##-> 2nd Convolution layer
    Conv2D(64, (5, 5), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    ##-> Flattening
    Flatten(),
    
    ##-> Fully Connected layer
    Dense(1024),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.4),
    Dense(num_classes, activation='softmax')
])

## Step 6: Schedule Learning Rate and Compile model

In [ ]:
initial_learning_rate = 0.005

lr_sched = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=initial_learning_rate, decay_steps=100, decay_rate=0.96, staircase=True)

model.compile(optimizer=Adam(learning_rate=lr_sched), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

## Step 7: CallBacks

In [ ]:
checkpoint = ModelCheckpoint("model_weights.h5", monitor='val_loss', save_weights_only=True, mode='min', verbose=1)
#? Add more callbacks like EarlyStopping?
callbacks = [PlotLossesCallback(), checkpoint]#, reduce_lr]

## Step 8: Model Training

In [ ]:
batch_size = 32

history = model.fit(
    dataGen_train.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=len(x_train) // batch_size,
    validation_data=dataGen_val.flow(x_val, y_val, batch_size=batch_size),
    validation_steps=len(x_val) // batch_size,
    epochs=12,
    callbacks=callbacks
)

## Step 9: Model Evaluation, Prediction, Classification Report

In [ ]:
model.evaluate(x_val, y_val)

y_true = np.argmax(y_val, axis=1)
y_pred = np.argmax(model.predict(x_val), axis=1)
print(metrics.classification_report(y_true, y_pred, zero_division=0))
print("Classification accuracy: %0.6f" % metrics.accuracy_score(y_true, y_pred))

##-> Visualize
labels = ["squiggle", "narrowband", "noise", "narrowbanddrd"]
confusion_mtx = metrics.confusion_matrix(y_true, y_pred)
sns.heatmap(confusion_mtx, annot=True, fmt='g', cmap='Blues')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.xticks(ticks=np.arange(num_classes), labels=labels)
plt.yticks(ticks=np.arange(num_classes), labels=labels)
plt.title('Confusion Matrix')
plt.show()